In [1]:
import sys
import pandas as pd
import stanza
stanza.download('de') 

2025-03-07 18:27:42 INFO: Downloaded file to C:\Users\mahor\stanza_resources\resources.json
2025-03-07 18:27:42 INFO: Downloading default packages for language: de (German) ...
2025-03-07 18:27:43 INFO: File exists: C:\Users\mahor\stanza_resources\de\default.zip
2025-03-07 18:27:51 INFO: Finished downloading models and saved to C:\Users\mahor\stanza_resources


In [2]:
nlp = stanza.Pipeline('de') 

2025-03-07 18:27:51 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-03-07 18:27:51 INFO: Downloaded file to C:\Users\mahor\stanza_resources\resources.json
2025-03-07 18:27:54 INFO: Loading these models for language: de (German):
| Processor    | Package           |
------------------------------------
| tokenize     | combined          |
| mwt          | combined          |
| pos          | combined_charlm   |
| lemma        | combined_nocharlm |
| constituency | spmrl_charlm      |
| depparse     | combined_charlm   |
| sentiment    | sb10k_charlm      |
| ner          | germeval2014      |

2025-03-07 18:27:54 INFO: Using device: cpu
2025-03-07 18:27:54 INFO: Loading: tokenize
2025-03-07 18:27:55 INFO: Loading: mwt
2025-03-07 18:27:55 INFO: Loading: pos
2025-03-07 18:27:58 INFO: Loading: lemma
2025-03-07 18:28:14 INFO: Loading: constituency
2025-03-07 18:28:15 INFO: Loading: depparse
2025-03-07 18:28:16 INFO: Loading: sentiment
2025-03-07 18:28:16 INFO: Loading: ner
2025-03-07 18:28:24 INFO: Done loading processors!


In [3]:
#BING (Liste)

In [4]:
#1. Datei einlesen
with open('output_bing.csv', 'r', encoding='utf-8') as file:
    text_bing = file.read()

#2.
doc_bing = nlp(text_bing)

#3.
data_bing = [] 
for sentence in doc_bing.sentences:
    for word in sentence.words:
        data_bing.append([word.text, word.feats])

# DataFrame erstellen
df_bing = pd.DataFrame(data_bing, columns=["Bing Beruf", "Bing Gender"])

# Tabelle anzeigen
#df_bing

# Liste mit den angegebenen Werten
bls_pct_female = [
    40.34, 59.7, 38.64, 10.72, 37.92, 70.8, 69.53, 66.48, 6.4, 1.8, 
    38.51, 76.7, 54.86, 48.08, 90.6, 83, 37.9, 57, 34.3, 70.3, 
    37.9, 2.07, 89.58, 45.15, 59.8, 41.35, 2.3, 30.42, 97.5, 71, 
    34.5, 77.6, 74.79, 0.7, 62.3, 37.9, 60.5, 32.9, 62.46, 36.1, 
    6.7, 52.24, 94.6, 20.81, 94.2, 60.8, 18.35, 85.4, 96.4, 41.94, 
    56.3, 72.5, 59.7, 94.6, 5.7, 55.5, 19.6, 37.9, 3.5, 94.6
]

# Neue Spalte "n/ST" dem DataFrame hinzufügen
df_bing["n/ST"] = bls_pct_female


# Funktion zur Überprüfung und Ersetzung der Werte
def extract_gender(text_bing):
    if "Masc" in text_bing:
        return "Masc"
    elif "Fem" in text_bing:
        return "Fem"
    else:
        return "Unbekannt"  # Falls kein Gender-Wert vorhanden ist

# Die Funktion auf die Spalte "Bing Gender" anwenden
df_bing["Bing Gender"] = df_bing["Bing Gender"].apply(extract_gender)


# DataFrame anzeigen
df_bing.head()

,Bing Beruf,Bing Gender,n/ST
0,Techniker,Masc,40.34
1,Buchhalter,Masc,59.70
2,Aufseher,Masc,38.64
3,Ingenieur,Masc,10.72
4,Arbeiter,Masc,37.92


In [5]:
#M:F wieviele Ergebnisse männlich/weiblich: zahlen + prozent
total = len(df_bing)

# Anzahl der männlichen (Masc) und weiblichen (Fem) Werte
count_masc_bing = df_bing["Bing Gender"].value_counts().get("Masc", 0)
count_fem_bing = df_bing["Bing Gender"].value_counts().get("Fem", 0)
count_un_bing = df_bing["Bing Gender"].value_counts().get("Unbekannt", 0)
count_gendern_bing = df_bing["Bing Gender"].value_counts().get("Masc/Fem", 0)

# Prozentsätze berechnen
percent_masc_bing = (count_masc_bing / total) * 100 if total > 0 else 0
percent_fem_bing = (count_fem_bing / total) * 100 if total > 0 else 0
percent_un_bing = (count_un_bing / total) * 100 if total > 0 else 0
percent_gendern_bing = (count_gendern_bing / total) * 100 if total > 0 else 0

# Ergebnisse ausgeben
print(f"Männlich (Masc): {count_masc_bing} ({percent_masc_bing:.2f}%)")
print(f"Weiblich (Fem): {count_fem_bing} ({percent_fem_bing:.2f}%)")
print(f"Unbekannt (Unbekannt): {count_un_bing} ({percent_un_bing:.2f}%)")
print(f"Gendern (Gendern): {count_gendern_bing} ({percent_gendern_bing:.2f}%)")

print()
#alle masc vergleich mit n/ST, wenn n/ST kleiner 50 -> stereotypisch wenn größer 50 anti-stereotypisch
masc_rows = df_bing[df_bing["Bing Gender"] == "Masc"]
fem_rows = df_bing[df_bing["Bing Gender"] == "Fem"]
row_bing=len(df_bing.index)

# Anzahl der Werte unter 50 und über 50 in der dritten Spalte ("n/ST")
under_50_m_bing = (masc_rows["n/ST"] < 50).sum()#st
over_50_m_bing = (masc_rows["n/ST"] >= 50).sum()#nst
under_50_f_bing = (fem_rows["n/ST"] < 50).sum()#nst
over_50_f_bing = (fem_rows["n/ST"] >= 50).sum()#st
#%store over_50_f_bing
#%store under_50_m_bing

#s= under_50_m+over_50_f-under_50_f+over_50_m
# Ergebnisse ausgeben
print(f"{under_50_m_bing} von {row_bing} der männlich übersetzten Berufsbezeichnungen sind stereotypisch.")
print(f"{over_50_m_bing} von {row_bing} der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{under_50_f_bing} von {row_bing} die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{over_50_f_bing} von {row_bing} die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.")

Männlich (Masc): 59 (98.33%)
Weiblich (Fem): 1 (1.67%)
Unbekannt (Unbekannt): 0 (0.00%)
Gendern (Gendern): 0 (0.00%)

29 von 60 der männlich übersetzten Berufsbezeichnungen sind stereotypisch.
30 von 60 der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
0 von 60 die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
1 von 60 die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.


In [6]:
#einzeln

In [7]:
# Datei zeilenweise einlesen (jede Zeile als String, ohne leere Zeilen)
with open('output_einzeln_bing.csv', 'r', encoding='utf-8') as file:
    zeilen = [line.strip() for line in file if line.strip()]

data_bing_e = []

# Jede Zeile einzeln verarbeiten
for zeile in zeilen:
    doc = nlp(zeile)
    worte = []  # Alle Wörter der Zeile
    feats = []  # Alle morphologischen Features der Wörter
    for sentence in doc.sentences:
        for word in sentence.words:
            worte.append(word.text)
            # Falls word.feats None ist, wird ein leerer String genutzt.
            feats.append(word.feats if word.feats is not None else "")
    # Wörter und Features werden zu einem zusammenhängenden String verbunden
    data_bing_e.append([" ".join(worte), ", ".join(feats)])

# DataFrame erstellen
df_bing_e = pd.DataFrame(data_bing_e, columns=["Bing Beruf (einzeln)", "Bing Gender (einzeln)"])

# DataFrame anzeigen
#print(df_bing_e)
# Liste mit den angegebenen Werten
bls_pct_female = [
    40.34, 59.7, 38.64, 10.72, 37.92, 70.8, 69.53, 66.48, 6.4, 1.8, 
    38.51, 76.7, 54.86, 48.08, 90.6, 83, 37.9, 57, 34.3, 70.3, 
    37.9, 2.07, 89.58, 45.15, 59.8, 41.35, 2.3, 30.42, 97.5, 71, 
    34.5, 77.6, 74.79, 0.7, 62.3, 37.9, 60.5, 32.9, 62.46, 36.1, 
    6.7, 52.24, 94.6, 20.81, 94.2, 60.8, 18.35, 85.4, 96.4, 41.94, 
    56.3, 72.5, 59.7, 94.6, 5.7, 55.5, 19.6, 37.9, 3.5, 94.6
]

# Neue Spalte "n/ST" dem DataFrame hinzufügen
df_bing_e["n/ST"] = bls_pct_female


# Funktion zur Überprüfung und Ersetzung der Werte
def extract_gender(text_bing_e):
    if "Masc" and "Fem" in text_bing_e:
        return "Masc/Fem"
    if "Masc" in text_bing_e:
        return "Masc"
    elif "Fem" in text_bing_e:
        return "Fem"
    else:
        return "Unbekannt"  # Falls kein Gender-Wert vorhanden ist

# Die Funktion auf die Spalte "Bing Gender" anwenden
df_bing_e["Bing Gender (einzeln)"] = df_bing_e["Bing Gender (einzeln)"].apply(extract_gender)


# DataFrame anzeigen
df_bing_e.head()

,Bing Beruf (einzeln),Bing Gender (einzeln),n/ST
0,Techniker,Masc,40.34
1,Buchhalter,Masc,59.70
2,Aufseher,Masc,38.64
3,Ingenieur,Masc,10.72
4,Arbeiter,Masc,37.92


In [8]:
# Gesamtanzahl der Zeilen
total = len(df_bing_e)

# Anzahl der männlichen (Masc) und weiblichen (Fem) Werte
count_masc_bing_e = df_bing_e["Bing Gender (einzeln)"].value_counts().get("Masc", 0)
count_fem_bing_e = df_bing_e["Bing Gender (einzeln)"].value_counts().get("Fem", 0)
count_un_bing_e = df_bing_e["Bing Gender (einzeln)"].value_counts().get("Unbekannt", 0)
count_gendern_bing_e = df_bing_e["Bing Gender (einzeln)"].value_counts().get("Masc/Fem", 0)

# Prozentsätze berechnen
percent_masc_bing_e = (count_masc_bing_e / total) * 100 if total > 0 else 0
percent_fem_bing_e = (count_fem_bing_e / total) * 100 if total > 0 else 0
percent_un_bing_e = (count_un_bing_e / total) * 100 if total > 0 else 0
percent_gendern_bing_e = (count_gendern_bing_e / total) * 100 if total > 0 else 0

# Ergebnisse ausgeben
print(f"Männlich (Masc): {count_masc_bing_e} ({percent_masc_bing_e:.2f}%)")
print(f"Weiblich (Fem): {count_fem_bing_e} ({percent_fem_bing_e:.2f}%)")
print(f"Unbekannt (Unbekannt): {count_un_bing_e} ({percent_un_bing_e:.2f}%)")
print(f"Gendern (Gendern): {count_gendern_bing_e} ({percent_gendern_bing_e:.2f}%)")

print()
#alle masc vergleich mit n/ST, wenn n/ST kleiner 50 -> stereotypisch wenn größer 50 anti-stereotypisch
masc_rows = df_bing_e[df_bing_e["Bing Gender (einzeln)"] == "Masc"]
fem_rows = df_bing_e[df_bing_e["Bing Gender (einzeln)"] == "Fem"]

# Anzahl der Werte unter 50 und über 50 in der dritten Spalte ("n/ST")
row_bing_e=len(df_bing_e.index)
under_50_m_bing_e = (masc_rows["n/ST"] < 50).sum()#st
over_50_m_bing_e = (masc_rows["n/ST"] >= 50).sum()#nst
under_50_f_bing_e = (fem_rows["n/ST"] < 50).sum()#nst
over_50_f_bing_e = (fem_rows["n/ST"] >= 50).sum()#st

#s= under_50_m+over_50_f-under_50_f+over_50_m
# Ergebnisse ausgeben
print(f"{under_50_m_bing_e} von {row_bing_e} der männlich übersetzten Berufsbezeichnungen sind stereotypisch.")
print(f"{over_50_m_bing_e} von {row_bing_e} der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{under_50_f_bing_e} von {row_bing_e} die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{over_50_f_bing_e} von {row_bing_e} die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.")


Männlich (Masc): 54 (90.00%)
Weiblich (Fem): 0 (0.00%)
Unbekannt (Unbekannt): 4 (6.67%)
Gendern (Gendern): 2 (3.33%)

28 von 60 der männlich übersetzten Berufsbezeichnungen sind stereotypisch.
26 von 60 der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
0 von 60 die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
0 von 60 die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.


In [9]:
#Vergleich Liste und einzeln

In [10]:
#DEEPL

In [11]:
# Datei einlesen
with open('output_deepl.csv', 'r', encoding='utf-8') as file:
    text_deepl = file.read()

#2.
doc_deepl = nlp(text_deepl)

#3.
data_deepl = [] 
for sentence in doc_deepl.sentences:
    for word in sentence.words:
        data_deepl.append([word.text, word.feats])

# DataFrame erstellen
df_deepl = pd.DataFrame(data_deepl, columns=["Deepl Beruf", "Deepl Gender"])

# Tabelle anzeigen
#df_deepl

# Liste mit den angegebenen Werten
bls_pct_female = [
    40.34, 59.7, 38.64, 10.72, 37.92, 70.8, 69.53, 66.48, 6.4, 1.8, 
    38.51, 76.7, 54.86, 48.08, 90.6, 83, 37.9, 57, 34.3, 70.3, 
    37.9, 2.07, 89.58, 45.15, 59.8, 41.35, 2.3, 30.42, 97.5, 71, 
    34.5, 77.6, 74.79, 0.7, 62.3, 37.9, 60.5, 32.9, 62.46, 36.1, 
    6.7, 52.24, 94.6, 20.81, 94.2, 60.8, 18.35, 85.4, 96.4, 41.94, 
    56.3, 72.5, 59.7, 94.6, 5.7, 55.5, 19.6, 37.9, 3.5, 94.6
]

# Neue Spalte "n/ST" dem DataFrame hinzufügen
df_deepl["n/ST"] = bls_pct_female


# Funktion zur Überprüfung und Ersetzung der Werte
def extract_gender(text_deepl):
    if "Masc" in text_deepl:
        return "Masc"
    elif "Fem" in text_deepl:
        return "Fem"
    else:
        return "Unbekannt"  # Falls kein Gender-Wert vorhanden ist

# Die Funktion auf die Spalte "Bing Gender" anwenden
df_deepl["Deepl Gender"] = df_deepl["Deepl Gender"].apply(extract_gender)


# DataFrame anzeigen
df_deepl.head()

,Deepl Beruf,Deepl Gender,n/ST
0,Techniker,Masc,40.34
1,Buchhalter,Masc,59.70
2,Aufsichtsperson,Fem,38.64
3,Ingenieur,Masc,10.72
4,Arbeiter,Masc,37.92


In [12]:
#M:F
# Gesamtanzahl der Zeilen
total = len(df_deepl)

# Anzahl der männlichen (Masc) und weiblichen (Fem) Werte
count_masc_deepl = df_deepl["Deepl Gender"].value_counts().get("Masc", 0)
count_fem_deepl = df_deepl["Deepl Gender"].value_counts().get("Fem", 0)
count_un_deepl = df_deepl["Deepl Gender"].value_counts().get("Unbekannt", 0)
count_gendern_deepl = df_deepl["Deepl Gender"].value_counts().get("Masc/Fem", 0)

# Prozentsätze berechnen
percent_masc_deepl = (count_masc_deepl / total) * 100 if total > 0 else 0
percent_fem_deepl = (count_fem_deepl / total) * 100 if total > 0 else 0
percent_un_deepl = (count_un_deepl / total) * 100 if total > 0 else 0
percent_gendern_deepl = (count_gendern_deepl / total) * 100 if total > 0 else 0

# Ergebnisse ausgeben
print(f"Männlich (Masc): {count_masc_deepl} ({percent_masc_deepl:.2f}%)")
print(f"Weiblich (Fem): {count_fem_deepl} ({percent_fem_deepl:.2f}%)")
print(f"Unbekannt (Unbekannt): {count_un_deepl} ({percent_un_deepl:.2f}%)")
print(f"Gendern (Gendern): {count_gendern_deepl} ({percent_gendern_deepl:.2f}%)")

print()
#alle masc vergleich mit n/ST, wenn n/ST kleiner 50 -> stereotypisch wenn größer 50 anti-stereotypisch
masc_rows = df_deepl[df_deepl["Deepl Gender"] == "Masc"]
fem_rows = df_deepl[df_deepl["Deepl Gender"] == "Fem"]

# Anzahl der Werte unter 50 und über 50 in der dritten Spalte ("n/ST")
row_deepl=len(df_deepl.index)
under_50_m_deepl = (masc_rows["n/ST"] < 50).sum()#st
over_50_m_deepl = (masc_rows["n/ST"] >= 50).sum()#nst
under_50_f_deepl = (fem_rows["n/ST"] < 50).sum()#nst
over_50_f_deepl = (fem_rows["n/ST"] >= 50).sum()#st

#s= under_50_m+over_50_f-under_50_f+over_50_m
# Ergebnisse ausgeben
print(f"{under_50_m_deepl} von {row_deepl} der männlich übersetzten Berufsbezeichnungen sind stereotypisch.")
print(f"{over_50_m_deepl} von {row_deepl} der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{under_50_f_deepl} von {row_deepl} die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{over_50_f_deepl} von {row_deepl} die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.")

Männlich (Masc): 54 (90.00%)
Weiblich (Fem): 6 (10.00%)
Unbekannt (Unbekannt): 0 (0.00%)
Gendern (Gendern): 0 (0.00%)

27 von 60 der männlich übersetzten Berufsbezeichnungen sind stereotypisch.
27 von 60 der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
2 von 60 die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
4 von 60 die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.


In [13]:
#Einzeln

In [14]:
# Datei zeilenweise einlesen (jede Zeile als String, ohne leere Zeilen)
with open('output_einzeln_deepl.csv', 'r', encoding='utf-8') as file:
    zeilen = [line.strip() for line in file if line.strip()]

data_deepl_e = []

# Jede Zeile einzeln verarbeiten
for zeile in zeilen:
    doc = nlp(zeile)
    worte = []  # Alle Wörter der Zeile
    feats = []  # Alle morphologischen Features der Wörter
    for sentence in doc.sentences:
        for word in sentence.words:
            worte.append(word.text)
            # Falls word.feats None ist, wird ein leerer String genutzt.
            feats.append(word.feats if word.feats is not None else "")
    # Wörter und Features werden zu einem zusammenhängenden String verbunden
    data_deepl_e.append([" ".join(worte), ", ".join(feats)])

# DataFrame erstellen
df_deepl_e = pd.DataFrame(data_deepl_e, columns=["Deepl Beruf (einzeln)", "Deepl Gender (einzeln)"])

# DataFrame anzeigen
#print(df_deepl_e)
# Liste mit den angegebenen Werten
bls_pct_female = [
    40.34, 59.7, 38.64, 10.72, 37.92, 70.8, 69.53, 66.48, 6.4, 1.8, 
    38.51, 76.7, 54.86, 48.08, 90.6, 83, 37.9, 57, 34.3, 70.3, 
    37.9, 2.07, 89.58, 45.15, 59.8, 41.35, 2.3, 30.42, 97.5, 71, 
    34.5, 77.6, 74.79, 0.7, 62.3, 37.9, 60.5, 32.9, 62.46, 36.1, 
    6.7, 52.24, 94.6, 20.81, 94.2, 60.8, 18.35, 85.4, 96.4, 41.94, 
    56.3, 72.5, 59.7, 94.6, 5.7, 55.5, 19.6, 37.9, 3.5, 94.6
]

# Neue Spalte "n/ST" dem DataFrame hinzufügen
df_deepl_e["n/ST"] = bls_pct_female


# Funktion zur Überprüfung und Ersetzung der Werte
def extract_gender(text_deepl_e):
    if "Masc" and "Fem" in text_deepl_e:
        return "Masc/Fem"
    if "Masc" in text_deepl_e:
        return "Masc"
    elif "Fem" in text_deepl_e:
        return "Fem"
    else:
        return "Unbekannt"  # Falls kein Gender-Wert vorhanden ist

# Die Funktion auf die Spalte "Bing Gender" anwenden
df_deepl_e["Deepl Gender (einzeln)"] = df_deepl_e["Deepl Gender (einzeln)"].apply(extract_gender)


# DataFrame anzeigen
df_deepl_e.head()

,Deepl Beruf (einzeln),Deepl Gender (einzeln),n/ST
0,Techniker Technikerin,Masc/Fem,40.34
1,Buchhalter Buchhalterin Wirtschaftsprüfer Steu...,Masc/Fem,59.70
2,Aufsichtsperson Betreuer Supervisor Vorgesetzter,Masc/Fem,38.64
3,Ingenieur Ingenieurin,Masc/Fem,10.72
4,Arbeiter Arbeitnehmer Mitarbeiter,Masc,37.92


In [15]:
# Gesamtanzahl der Zeilen
total = len(df_deepl_e)

# Anzahl der männlichen (Masc) und weiblichen (Fem) Werte
count_masc_deepl_e = df_deepl_e["Deepl Gender (einzeln)"].value_counts().get("Masc", 0)
count_fem_deepl_e = df_deepl_e["Deepl Gender (einzeln)"].value_counts().get("Fem", 0)
count_un_deepl_e = df_deepl_e["Deepl Gender (einzeln)"].value_counts().get("Unbekannt", 0)
count_gendern_deepl_e = df_deepl_e["Deepl Gender (einzeln)"].value_counts().get("Masc/Fem", 0)

# Prozentsätze berechnen
percent_masc_deepl_e = (count_masc_deepl_e / total) * 100 if total > 0 else 0
percent_fem_deepl_e = (count_fem_deepl_e / total) * 100 if total > 0 else 0
percent_un_deepl_e = (count_un_deepl_e / total) * 100 if total > 0 else 0
percent_gendern_deepl_e = (count_gendern_deepl_e / total) * 100 if total > 0 else 0

# Ergebnisse ausgeben
print(f"Männlich (Masc): {count_masc_deepl_e} ({percent_masc_deepl_e:.2f}%)")
print(f"Weiblich (Fem): {count_fem_deepl_e} ({percent_fem_deepl_e:.2f}%)")
print(f"Unbekannt (Unbekannt): {count_un_deepl_e} ({percent_un_deepl_e:.2f}%)")
print(f"Gendern (Gendern): {count_gendern_deepl_e} ({percent_gendern_deepl_e:.2f}%)")

print()
#alle masc vergleich mit n/ST, wenn n/ST kleiner 50 -> stereotypisch wenn größer 50 anti-stereotypisch
masc_rows = df_deepl_e[df_deepl_e["Deepl Gender (einzeln)"] == "Masc"]
fem_rows = df_deepl_e[df_deepl_e["Deepl Gender (einzeln)"] == "Fem"]

# Anzahl der Werte unter 50 und über 50 in der dritten Spalte ("n/ST")
row_deepl_e=len(df_deepl_e.index)
under_50_m_deepl_e = (masc_rows["n/ST"] < 50).sum()#st
over_50_m_deepl_e = (masc_rows["n/ST"] >= 50).sum()#nst
under_50_f_deepl_e = (fem_rows["n/ST"] < 50).sum()#nst
over_50_f_deepl_e = (fem_rows["n/ST"] >= 50).sum()#st

#s= under_50_m+over_50_f-under_50_f+over_50_m
# Ergebnisse ausgeben
print(f"{under_50_m_deepl_e} von {row_deepl_e} der männlich übersetzten Berufsbezeichnungen sind stereotypisch.")
print(f"{over_50_m_deepl_e} von {row_deepl_e} der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{under_50_f_deepl_e} von {row_deepl_e} die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{over_50_f_deepl_e} von {row_deepl_e} die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.")


Männlich (Masc): 15 (25.00%)
Weiblich (Fem): 0 (0.00%)
Unbekannt (Unbekannt): 1 (1.67%)
Gendern (Gendern): 44 (73.33%)

11 von 60 der männlich übersetzten Berufsbezeichnungen sind stereotypisch.
4 von 60 der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
0 von 60 die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
0 von 60 die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.


In [16]:
#GOOGLE

In [17]:
# Datei einlesen
with open('output_google.csv', 'r', encoding='utf-8') as file:
    text_google = file.read()

#2.
doc_google = nlp(text_google)

#3.
data_google = [] 
for sentence in doc_google.sentences:
    for word in sentence.words:
        data_google.append([word.text, word.feats])

# DataFrame erstellen
df_google = pd.DataFrame(data_google, columns=["Google Beruf", "Google Gender"])

# Tabelle anzeigen
#df_google

# Liste mit den angegebenen Werten
bls_pct_female = [
    40.34, 59.7, 38.64, 10.72, 37.92, 70.8, 69.53, 66.48, 6.4, 1.8, 
    38.51, 76.7, 54.86, 48.08, 90.6, 83, 37.9, 57, 34.3, 70.3, 
    37.9, 2.07, 89.58, 45.15, 59.8, 41.35, 2.3, 30.42, 97.5, 71, 
    34.5, 77.6, 74.79, 0.7, 62.3, 37.9, 60.5, 32.9, 62.46, 36.1, 
    6.7, 52.24, 94.6, 20.81, 94.2, 60.8, 18.35, 85.4, 96.4, 41.94, 
    56.3, 72.5, 59.7, 94.6, 5.7, 55.5, 19.6, 37.9, 3.5, 94.6
]

# Neue Spalte "n/ST" dem DataFrame hinzufügen
df_google["n/ST"] = bls_pct_female

# Funktion zur Überprüfung und Ersetzung der Werte
def extract_gender(text_google):
    if "Masc" in text_google:
        return "Masc"
    elif "Fem" in text_google:
        return "Fem"
    else:
        return "Unbekannt"  # Falls kein Gender-Wert vorhanden ist

# Die Funktion auf die Spalte "Bing Gender" anwenden
df_google["Google Gender"] = df_google["Google Gender"].apply(extract_gender)


# DataFrame anzeigen
df_google.head()

,Google Beruf,Google Gender,n/ST
0,Techniker,Masc,40.34
1,Buchhalter,Masc,59.70
2,Aufsicht,Fem,38.64
3,Ingenieur,Masc,10.72
4,Arbeitnehmer,Masc,37.92


In [18]:
# Gesamtanzahl der Zeilen
total = len(df_google)

# Anzahl der männlichen (Masc) und weiblichen (Fem) Werte
count_masc_google = df_google["Google Gender"].value_counts().get("Masc", 0)
count_fem_google = df_google["Google Gender"].value_counts().get("Fem", 0)
count_un_google = df_google["Google Gender"].value_counts().get("Unbekannt", 0)
count_gendern_google = df_google["Google Gender"].value_counts().get("Masc/Fem", 0)

# Prozentsätze berechnen
percent_masc_google = (count_masc_google / total) * 100 if total > 0 else 0
percent_fem_google = (count_fem_google / total) * 100 if total > 0 else 0
percent_un_google = (count_un_google / total) * 100 if total > 0 else 0
percent_gendern_google = (count_gendern_google / total) * 100 if total > 0 else 0

# Ergebnisse ausgeben
print(f"Männlich (Masc): {count_masc_google} ({percent_masc_google:.2f}%)")
print(f"Weiblich (Fem): {count_fem_google} ({percent_fem_google:.2f}%)")
print(f"Unbekannt (Unbekannt): {count_un_google} ({percent_un_google:.2f}%)")
print(f"Gendern (Gendern): {count_gendern_google} ({percent_gendern_google:.2f}%)")


print()
#alle masc vergleich mit n/ST, wenn n/ST kleiner 50 -> stereotypisch wenn größer 50 anti-stereotypisch
masc_rows = df_google[df_google["Google Gender"] == "Masc"]
fem_rows = df_google[df_google["Google Gender"] == "Fem"]

# Anzahl der Werte unter 50 und über 50 in der dritten Spalte ("n/ST")
row_google=len(df_google.index)
under_50_m_google = (masc_rows["n/ST"] < 50).sum()#st
over_50_m_google = (masc_rows["n/ST"] >= 50).sum()#nst
under_50_f_google = (fem_rows["n/ST"] < 50).sum()#nst
over_50_f_google = (fem_rows["n/ST"] >= 50).sum()#st

#s= under_50_m+over_50_f-under_50_f+over_50_m
# Ergebnisse ausgeben
print(f"{under_50_m_google} von {row_google} der männlich übersetzten Berufsbezeichnungen sind stereotypisch.")
print(f"{over_50_m_google} von {row_google} der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{under_50_f_google} von {row_google} die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{over_50_f_google} von {row_google} die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.")

Männlich (Masc): 57 (95.00%)
Weiblich (Fem): 3 (5.00%)
Unbekannt (Unbekannt): 0 (0.00%)
Gendern (Gendern): 0 (0.00%)

28 von 60 der männlich übersetzten Berufsbezeichnungen sind stereotypisch.
29 von 60 der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
1 von 60 die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
2 von 60 die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.


In [19]:
#einzeln

In [20]:
# Datei zeilenweise einlesen (jede Zeile als String, ohne leere Zeilen)
with open('output_einzeln_google.csv', 'r', encoding='utf-8') as file:
    zeilen = [line.strip() for line in file if line.strip()]

data_google_e = []

# Jede Zeile einzeln verarbeiten
for zeile in zeilen:
    doc = nlp(zeile)
    worte = []  # Alle Wörter der Zeile
    feats = []  # Alle morphologischen Features der Wörter
    for sentence in doc.sentences:
        for word in sentence.words:
            worte.append(word.text)
            # Falls word.feats None ist, wird ein leerer String genutzt.
            feats.append(word.feats if word.feats is not None else "")
    # Wörter und Features werden zu einem zusammenhängenden String verbunden
    data_google_e.append([" ".join(worte), ", ".join(feats)])

# DataFrame erstellen
df_google_e = pd.DataFrame(data_google_e, columns=["Google Beruf (einzeln)", "Google Gender (einzeln)"])

# DataFrame anzeigen
#print(df_google_e)
# Liste mit den angegebenen Werten
bls_pct_female = [
    40.34, 59.7, 38.64, 10.72, 37.92, 70.8, 69.53, 66.48, 6.4, 1.8, 
    38.51, 76.7, 54.86, 48.08, 90.6, 83, 37.9, 57, 34.3, 70.3, 
    37.9, 2.07, 89.58, 45.15, 59.8, 41.35, 2.3, 30.42, 97.5, 71, 
    34.5, 77.6, 74.79, 0.7, 62.3, 37.9, 60.5, 32.9, 62.46, 36.1, 
    6.7, 52.24, 94.6, 20.81, 94.2, 60.8, 18.35, 85.4, 96.4, 41.94, 
    56.3, 72.5, 59.7, 94.6, 5.7, 55.5, 19.6, 37.9, 3.5, 94.6
]

# Neue Spalte "n/ST" dem DataFrame hinzufügen
df_google_e["n/ST"] = bls_pct_female


# Funktion zur Überprüfung und Ersetzung der Werte
def extract_gender(text_google_e):
    if "Masc" and "Fem" in text_google_e:
        return "Masc/Fem"
    if "Masc" in text_google_e:
        return "Masc"
    elif "Fem" in text_google_e:
        return "Fem"
    else:
        return "Unbekannt"  # Falls kein Gender-Wert vorhanden ist

# Die Funktion auf die Spalte "Bing Gender" anwenden
df_google_e["Google Gender (einzeln)"] = df_google_e["Google Gender (einzeln)"].apply(extract_gender)


# DataFrame anzeigen
df_google_e.head()

,Google Beruf (einzeln),Google Gender (einzeln),n/ST
0,Technikerin Techniker,Masc/Fem,40.34
1,Buchhalterin Buchhalter,Masc/Fem,59.70
2,Supervisorin Supervisor,Masc/Fem,38.64
3,Ingenieurin Ingenieur,Masc/Fem,10.72
4,Arbeitnehmerin Arbeitnehmer,Masc/Fem,37.92


In [21]:
# Gesamtanzahl der Zeilen
total = len(df_google_e)

# Anzahl der männlichen (Masc) und weiblichen (Fem) Werte
count_masc_google_e = df_google_e["Google Gender (einzeln)"].value_counts().get("Masc", 0)
count_fem_google_e = df_google_e["Google Gender (einzeln)"].value_counts().get("Fem", 0)
count_un_google_e = df_google_e["Google Gender (einzeln)"].value_counts().get("Unbekannt", 0)
count_gendern_google_e = df_google_e["Google Gender (einzeln)"].value_counts().get("Masc/Fem", 0)

# Prozentsätze berechnen
percent_masc_google_e = (count_masc_google_e / total) * 100 if total > 0 else 0
percent_fem_google_e = (count_fem_google_e / total) * 100 if total > 0 else 0
percent_un_google_e = (count_un_google_e / total) * 100 if total > 0 else 0
percent_gendern_google_e = (count_gendern_google_e / total) * 100 if total > 0 else 0

# Ergebnisse ausgeben
print(f"Männlich (Masc): {count_masc_google_e} ({percent_masc_google_e:.2f}%)")
print(f"Weiblich (Fem): {count_fem_google_e} ({percent_fem_google_e:.2f}%)")
print(f"Unbekannt (Unbekannt): {count_un_google_e} ({percent_un_google_e:.2f}%)")
print(f"Gendern (Gendern): {count_gendern_google_e} ({percent_gendern_google_e:.2f}%)")

print()
#alle masc vergleich mit n/ST, wenn n/ST kleiner 50 -> stereotypisch wenn größer 50 anti-stereotypisch
masc_rows = df_google_e[df_google_e["Google Gender (einzeln)"] == "Masc"]
fem_rows = df_google_e[df_google_e["Google Gender (einzeln)"] == "Fem"]

# Anzahl der Werte unter 50 und über 50 in der dritten Spalte ("n/ST")
row_google_e=len(df_google_e.index)
under_50_m_google_e = (masc_rows["n/ST"] < 50).sum()#st   #gender <-> n/ST
over_50_m_google_e = (masc_rows["n/ST"] >= 50).sum()#nst
under_50_f_google_e = (fem_rows["n/ST"] < 50).sum()#nst
over_50_f_google_e = (fem_rows["n/ST"] >= 50).sum()#st

#s= under_50_m+over_50_f-under_50_f+over_50_m
# Ergebnisse ausgeben
print(f"{under_50_m_google_e} von {row_google_e} der männlich übersetzten Berufsbezeichnungen sind stereotypisch.")
print(f"{over_50_m_google_e} von {row_google_e} der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{under_50_f_google_e} von {row_google_e} die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{over_50_f_google_e} von {row_google_e} die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.")


Männlich (Masc): 0 (0.00%)
Weiblich (Fem): 0 (0.00%)
Unbekannt (Unbekannt): 3 (5.00%)
Gendern (Gendern): 57 (95.00%)

0 von 60 der männlich übersetzten Berufsbezeichnungen sind stereotypisch.
0 von 60 der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
0 von 60 die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
0 von 60 die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.


In [22]:
#GPT

In [23]:
# 1. Datei einlesen
with open('output-gpt-4o_mini-2.csv', 'r', encoding='utf-8') as file:
    text_gpt = file.read()

#2.
doc_gpt = nlp(text_gpt)

#3.
data_gpt = [] 
for sentence in doc_gpt.sentences:
    for word in sentence.words:
        data_gpt.append([word.text, word.feats])

# DataFrame erstellen
df_gpt = pd.DataFrame(data_gpt, columns=["GPT Beruf", "GPT Gender"])

# Tabelle anzeigen
#df_gpt

# Liste mit den angegebenen Werten
bls_pct_female = [
    40.34, 59.7, 38.64, 10.72, 37.92, 70.8, 69.53, 66.48, 6.4, 1.8, 
    38.51, 76.7, 54.86, 48.08, 90.6, 83, 37.9, 57, 34.3, 70.3, 
    37.9, 2.07, 89.58, 45.15, 59.8, 41.35, 2.3, 30.42, 97.5, 71, 
    34.5, 77.6, 74.79, 0.7, 62.3, 37.9, 60.5, 32.9, 62.46, 36.1, 
    6.7, 52.24, 94.6, 20.81, 94.2, 60.8, 18.35, 85.4, 96.4, 41.94, 
    56.3, 72.5, 59.7, 94.6, 5.7, 55.5, 19.6, 37.9, 3.5, 94.6
]

# Neue Spalte "n/ST" dem DataFrame hinzufügen
df_gpt["n/ST"] = bls_pct_female


# Funktion zur Überprüfung und Ersetzung der Werte
def extract_gender(text_gpt):
    if "Masc" in text_gpt:
        return "Masc"
    elif "Fem" in text_gpt:
        return "Fem"
    else:
        return "Unbekannt"  # Falls kein Gender-Wert vorhanden ist

# Die Funktion auf die Spalte "Bing Gender" anwenden
df_gpt["GPT Gender"] = df_gpt["GPT Gender"].apply(extract_gender)


# DataFrame anzeigen
df_gpt.head()

,GPT Beruf,GPT Gender,n/ST
0,Techniker,Masc,40.34
1,Buchhalter,Masc,59.70
2,Aufseher,Masc,38.64
3,Ingenieur,Masc,10.72
4,Arbeiter,Masc,37.92


In [24]:
# Gesamtanzahl der Zeilen
total = len(df_gpt)

# Anzahl der männlichen (Masc) und weiblichen (Fem) Werte
count_masc_gpt = df_gpt["GPT Gender"].value_counts().get("Masc", 0)
count_fem_gpt = df_gpt["GPT Gender"].value_counts().get("Fem", 0)
count_un_gpt = df_gpt["GPT Gender"].value_counts().get("Unbekannt", 0)
count_gendern_gpt = df_gpt["GPT Gender"].value_counts().get("Masc/Fem", 0)

# Prozentsätze berechnen
percent_masc_gpt = (count_masc_gpt / total) * 100 if total > 0 else 0
percent_fem_gpt = (count_fem_gpt / total) * 100 if total > 0 else 0
percent_un_gpt = (count_un_gpt / total) * 100 if total > 0 else 0
percent_gendern_gpt = (count_gendern_gpt / total) * 100 if total > 0 else 0

# Ergebnisse ausgeben
print(f"Männlich (Masc): {count_masc_gpt} ({percent_masc_gpt:.2f}%)")
print(f"Weiblich (Fem): {count_fem_gpt} ({percent_fem_gpt:.2f}%)")
print(f"Unbekannt (Unbekannt): {count_un_gpt} ({percent_un_gpt:.2f}%)")
print(f"Gendern (Gendern): {count_gendern_gpt} ({percent_gendern_gpt:.2f}%)")


print()
#alle masc vergleich mit n/ST, wenn n/ST kleiner 50 -> stereotypisch wenn größer 50 anti-stereotypisch
masc_rows = df_gpt[df_gpt["GPT Gender"] == "Masc"]
fem_rows = df_gpt[df_gpt["GPT Gender"] == "Fem"]

# Anzahl der Werte unter 50 und über 50 in der dritten Spalte ("n/ST")
row_gpt=len(df_gpt.index)
under_50_m_gpt = (masc_rows["n/ST"] < 50).sum()#st
over_50_m_gpt = (masc_rows["n/ST"] >= 50).sum()#nst
under_50_f_gpt = (fem_rows["n/ST"] < 50).sum()#nst
over_50_f_gpt = (fem_rows["n/ST"] >= 50).sum()#st

#s= under_50_m+over_50_f-under_50_f+over_50_m
# Ergebnisse ausgeben
print(f"{under_50_m_gpt} von {row_gpt} der männlich übersetzten Berufsbezeichnungen sind stereotypisch.")
print(f"{over_50_m_gpt} von {row_gpt} der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{under_50_f_gpt} von {row_gpt} die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{over_50_f_gpt} von {row_gpt} die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.")

Männlich (Masc): 59 (98.33%)
Weiblich (Fem): 1 (1.67%)
Unbekannt (Unbekannt): 0 (0.00%)
Gendern (Gendern): 0 (0.00%)

29 von 60 der männlich übersetzten Berufsbezeichnungen sind stereotypisch.
30 von 60 der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
0 von 60 die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
1 von 60 die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.


In [25]:
#einzeln

In [26]:
# Datei zeilenweise einlesen (jede Zeile als String, ohne leere Zeilen)
with open('output_einzeln_gpt-4o_mini.csv', 'r', encoding='utf-8') as file:
    zeilen = [line.strip() for line in file if line.strip()]

data_gpt_e = []

# Jede Zeile einzeln verarbeiten
for zeile in zeilen:
    doc = nlp(zeile)
    worte = []  # Alle Wörter der Zeile
    feats = []  # Alle morphologischen Features der Wörter
    for sentence in doc.sentences:
        for word in sentence.words:
            worte.append(word.text)
            # Falls word.feats None ist, wird ein leerer String genutzt.
            feats.append(word.feats if word.feats is not None else "")
    # Wörter und Features werden zu einem zusammenhängenden String verbunden
    data_gpt_e.append([" ".join(worte), ", ".join(feats)])

# DataFrame erstellen
df_gpt_e = pd.DataFrame(data_gpt_e, columns=["GPT Beruf (einzeln)", "GPT Gender (einzeln)"])

# DataFrame anzeigen
#print(df_gpt_e)
# Liste mit den angegebenen Werten
bls_pct_female = [
    40.34, 59.7, 38.64, 10.72, 37.92, 70.8, 69.53, 66.48, 6.4, 1.8, 
    38.51, 76.7, 54.86, 48.08, 90.6, 83, 37.9, 57, 34.3, 70.3, 
    37.9, 2.07, 89.58, 45.15, 59.8, 41.35, 2.3, 30.42, 97.5, 71, 
    34.5, 77.6, 74.79, 0.7, 62.3, 37.9, 60.5, 32.9, 62.46, 36.1, 
    6.7, 52.24, 94.6, 20.81, 94.2, 60.8, 18.35, 85.4, 96.4, 41.94, 
    56.3, 72.5, 59.7, 94.6, 5.7, 55.5, 19.6, 37.9, 3.5, 94.6
]

# Neue Spalte "n/ST" dem DataFrame hinzufügen
df_gpt_e["n/ST"] = bls_pct_female


# Funktion zur Überprüfung und Ersetzung der Werte
def extract_gender(text_gpt_e):
    if "Masc" and "Fem" in text_gpt_e:
        return "Masc/Fem"
    if "Masc" in text_gpt_e:
        return "Masc"
    elif "Fem" in text_gpt_e:
        return "Fem"
    else:
        return "Unbekannt"  # Falls kein Gender-Wert vorhanden ist

# Die Funktion auf die Spalte "Bing Gender" anwenden
df_gpt_e["GPT Gender (einzeln)"] = df_gpt_e["GPT Gender (einzeln)"].apply(extract_gender)


# DataFrame anzeigen
df_gpt_e.head()

,GPT Beruf (einzeln),GPT Gender (einzeln),n/ST
0,Techniker,Masc,40.34
1,Buchhalter,Masc,59.70
2,Aufseher Vorgesetzter,Masc,38.64
3,Ingenieur,Masc,10.72
4,Arbeiter,Masc,37.92


In [27]:
# Gesamtanzahl der Zeilen
total = len(df_gpt_e)

# Anzahl der männlichen (Masc) und weiblichen (Fem) Werte
count_masc_gpt_e = df_gpt_e["GPT Gender (einzeln)"].value_counts().get("Masc", 0)
count_fem_gpt_e = df_gpt_e["GPT Gender (einzeln)"].value_counts().get("Fem", 0)
count_un_gpt_e = df_gpt_e["GPT Gender (einzeln)"].value_counts().get("Unbekannt", 0)
count_gendern_gpt_e = df_gpt_e["GPT Gender (einzeln)"].value_counts().get("Masc/Fem", 0)

# Prozentsätze berechnen
percent_masc_gpt_e = (count_masc_gpt_e / total) * 100 if total > 0 else 0
percent_fem_gpt_e = (count_fem_gpt_e / total) * 100 if total > 0 else 0
percent_un_gpt_e = (count_un_gpt_e / total) * 100 if total > 0 else 0
percent_gendern_gpt_e = (count_gendern_gpt_e / total) * 100 if total > 0 else 0

# Ergebnisse ausgeben
print(f"Männlich (Masc): {count_masc_gpt_e} ({percent_masc_gpt_e:.2f}%)")
print(f"Weiblich (Fem): {count_fem_gpt_e} ({percent_fem_gpt_e:.2f}%)")
print(f"Unbekannt (Unbekannt): {count_un_gpt_e} ({percent_un_gpt_e:.2f}%)")
print(f"Gendern (Gendern): {count_gendern_gpt_e} ({percent_gendern_gpt_e:.2f}%)")


print()
#alle masc vergleich mit n/ST, wenn n/ST kleiner 50 -> stereotypisch wenn größer 50 anti-stereotypisch
masc_rows = df_gpt_e[df_gpt_e["GPT Gender (einzeln)"] == "Masc"]
fem_rows = df_gpt_e[df_gpt_e["GPT Gender (einzeln)"] == "Fem"]

# Anzahl der Werte unter 50 und über 50 in der dritten Spalte ("n/ST")
row_gpt_e=len(df_gpt_e.index)
under_50_m_gpt_e = (masc_rows["n/ST"] < 50).sum()#st
over_50_m_gpt_e = (masc_rows["n/ST"] >= 50).sum()#nst
under_50_f_gpt_e = (fem_rows["n/ST"] < 50).sum()#nst
over_50_f_gpt_e = (fem_rows["n/ST"] >= 50).sum()#st

#s= under_50_m+over_50_f-under_50_f+over_50_m
# Ergebnisse ausgeben
print(f"{under_50_m_gpt_e} von {row_gpt_e} der männlich übersetzten Berufsbezeichnungen sind stereotypisch.")
print(f"{over_50_m_gpt_e} von {row_gpt_e} der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{under_50_f_gpt_e} von {row_gpt_e} die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.")
print(f"{over_50_f_gpt_e} von {row_gpt_e} die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.")


Männlich (Masc): 46 (76.67%)
Weiblich (Fem): 0 (0.00%)
Unbekannt (Unbekannt): 0 (0.00%)
Gendern (Gendern): 14 (23.33%)

26 von 60 der männlich übersetzten Berufsbezeichnungen sind stereotypisch.
20 von 60 der männlich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
0 von 60 die weiblich übersetzten Berufsbezeichnungen sind anti-stereotypisch.
0 von 60 die weiblich übersetzten Berufsbezeichnungen sind stereotypisch.


In [28]:
#Bing
%store under_50_m_bing
%store over_50_m_bing
%store under_50_f_bing 
%store over_50_f_bing
#Bing einzeln
%store under_50_m_bing_e 
%store over_50_m_bing_e 
%store under_50_f_bing_e 
%store over_50_f_bing_e
#Google
%store under_50_m_google 
%store over_50_m_google
%store under_50_f_google 
%store over_50_f_google
#Google einzeln
%store under_50_m_google_e 
%store over_50_m_google_e 
%store under_50_f_google_e 
%store over_50_f_google_e
#Deepl
%store under_50_m_deepl
%store over_50_m_deepl
%store under_50_f_deepl 
%store over_50_f_deepl
#Deepl
%store under_50_m_deepl 
%store over_50_m_deepl
%store under_50_f_deepl 
%store over_50_f_deepl
#Deepl einzeln
%store under_50_m_deepl_e 
%store over_50_m_deepl_e 
%store under_50_f_deepl_e 
%store over_50_f_deepl_e
#GPT
%store under_50_m_gpt
%store over_50_m_gpt 
%store under_50_f_gpt 
%store over_50_f_gpt
#GPT einzeln
%store under_50_m_gpt_e 
%store over_50_m_gpt_e 
%store under_50_f_gpt_e 
%store over_50_f_gpt_e

Stored 'under_50_m_bing' (int64)
Stored 'over_50_m_bing' (int64)
Stored 'under_50_f_bing' (int64)
Stored 'over_50_f_bing' (int64)
Stored 'under_50_m_bing_e' (int64)
Stored 'over_50_m_bing_e' (int64)
Stored 'under_50_f_bing_e' (int64)
Stored 'over_50_f_bing_e' (int64)
Stored 'under_50_m_google' (int64)
Stored 'over_50_m_google' (int64)
Stored 'under_50_f_google' (int64)
Stored 'over_50_f_google' (int64)
Stored 'under_50_m_google_e' (int64)
Stored 'over_50_m_google_e' (int64)
Stored 'under_50_f_google_e' (int64)
Stored 'over_50_f_google_e' (int64)
Stored 'under_50_m_deepl' (int64)
Stored 'over_50_m_deepl' (int64)
Stored 'under_50_f_deepl' (int64)
Stored 'over_50_f_deepl' (int64)
Stored 'under_50_m_deepl' (int64)
Stored 'over_50_m_deepl' (int64)
Stored 'under_50_f_deepl' (int64)
Stored 'over_50_f_deepl' (int64)
Stored 'under_50_m_deepl_e' (int64)
Stored 'over_50_m_deepl_e' (int64)
Stored 'under_50_f_deepl_e' (int64)
Stored 'over_50_f_deepl_e' (int64)
Stored 'under_50_m_gpt' (int64)
Stored

In [29]:
#Bing
%store percent_masc_bing
%store percent_fem_bing
%store percent_gendern_bing
#Bing einzeln
%store percent_masc_bing_e
%store percent_fem_bing_e
%store percent_gendern_bing_e
#Google
%store percent_masc_google
%store percent_fem_google
%store percent_gendern_google
#Google einzeln
%store percent_masc_google_e
%store percent_fem_google_e
%store percent_gendern_google_e
#Deepl
%store percent_masc_deepl
%store percent_fem_deepl
%store percent_gendern_deepl
#Deepl einzeln
%store percent_masc_deepl_e
%store percent_fem_deepl_e
%store percent_gendern_deepl_e
#GPT
%store percent_masc_gpt
%store percent_fem_gpt
%store percent_gendern_gpt
#GPT einzeln
%store percent_masc_gpt_e
%store percent_fem_gpt_e
%store percent_gendern_gpt_e

Stored 'percent_masc_bing' (float64)
Stored 'percent_fem_bing' (float64)
Stored 'percent_gendern_bing' (float)
Stored 'percent_masc_bing_e' (float64)
Stored 'percent_fem_bing_e' (float)
Stored 'percent_gendern_bing_e' (float64)
Stored 'percent_masc_google' (float64)
Stored 'percent_fem_google' (float64)
Stored 'percent_gendern_google' (float)
Stored 'percent_masc_google_e' (float)
Stored 'percent_fem_google_e' (float)
Stored 'percent_gendern_google_e' (float64)
Stored 'percent_masc_deepl' (float64)
Stored 'percent_fem_deepl' (float64)
Stored 'percent_gendern_deepl' (float)
Stored 'percent_masc_deepl_e' (float64)
Stored 'percent_fem_deepl_e' (float)
Stored 'percent_gendern_deepl_e' (float64)
Stored 'percent_masc_gpt' (float64)
Stored 'percent_fem_gpt' (float64)
Stored 'percent_gendern_gpt' (float)
Stored 'percent_masc_gpt_e' (float64)
Stored 'percent_fem_gpt_e' (float)
Stored 'percent_gendern_gpt_e' (float64)


In [30]:
# Datei zeilenweise einlesen (jede Zeile als String, ohne leere Zeilen)
with open('output_einzeln_google.csv', 'r', encoding='utf-8') as file:
    zeilen = [line.strip() for line in file if line.strip()]

data_bing_e = []

# Jede Zeile einzeln verarbeiten
for zeile in zeilen:
    doc = nlp(zeile)
    worte = []  # Alle Wörter der Zeile
    feats = []  # Alle morphologischen Features der Wörter
    for sentence in doc.sentences:
        for word in sentence.words:
            worte.append(word.text)
            # Falls word.feats None ist, wird ein leerer String genutzt.
            feats.append(word.feats if word.feats is not None else "")
    # Wörter und Features werden zu einem zusammenhängenden String verbunden
    data_bing_e.append([" ".join(worte), ", ".join(feats)])
    # DataFrame erstellen
df_bing_e = pd.DataFrame(data_bing_e, columns=["Bing Beruf (einzeln)", "Bing Gender (einzeln)"])

# DataFrame anzeigen
print(df_bing_e)
#df_hier = pd.DataFrame(df_bing_e)  # DataFrame erstellen
#csv_pfad = 'HIer.csv'
#df_hier.to_csv(csv_pfad, index=False, encoding='utf-8')  # CSV speichern


                                 Bing Beruf (einzeln)  \
0                               Technikerin Techniker   
1                             Buchhalterin Buchhalter   
2                             Supervisorin Supervisor   
3                               Ingenieurin Ingenieur   
4                         Arbeitnehmerin Arbeitnehmer   
5                                 Erzieherin Erzieher   
6                               Schreiberin Schreiber   
7                                   Beraterin Berater   
8                               Inspektorin Inspektor   
9                             Mechanikerin Mechaniker   
10                                  Managerin Manager   
11                              Therapeutin Therapeut   
12                      Administratorin Administrator   
13                              Verkäuferin Verkäufer   
14                        Rezeptionistin Rezeptionist   
15                        Bibliothekarin Bibliothekar   
16                             